In [1]:
import sqlite3
import pandas as pd

In [45]:
#Conectar a la base de datos SQLite
conexion = sqlite3.connect('sanoyfresco.db')

In [47]:
#Cargar tabla completa en un DataFrame
df = pd.read_sql_query('SELECT * FROM tickets', conexion)

#Mostrar los primeros registros del DataFrame
df.head()

,id_pedido,id_cliente,fecha,hora,id_departamento,id_seccion,id_producto,nombre_producto,precio_unitario,cantidad,precio_total
0,1,112108,2023-02-02 00:00:00,10,4,83,49683,Pepino Kirby,0.99,4,3.96
1,1,112108,2023-02-02 00:00:00,10,4,24,13176,Bolsa de Bananas Orgánicas,2.45,4,9.80
2,1,112108,2023-02-02 00:00:00,10,4,24,47209,Aguacate Hass Orgánico,1.79,2,3.58
3,2,202279,2023-02-03 00:00:00,9,4,83,28985,Col Rizada Orgánica de Michigan,2.55,2,5.10
4,2,202279,2023-02-03 00:00:00,9,4,83,17794,Zanahorias,0.95,5,4.75


In [49]:
#Cerrar conexion a la base de datos
conexion.close()

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975718 entries, 0 to 4975717
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id_pedido        int64  
 1   id_cliente       int64  
 2   fecha            object 
 3   hora             int64  
 4   id_departamento  int64  
 5   id_seccion       int64  
 6   id_producto      int64  
 7   nombre_producto  object 
 8   precio_unitario  float64
 9   cantidad         int64  
 10  precio_total     float64
dtypes: float64(2), int64(7), object(2)
memory usage: 417.6+ MB


In [53]:
#Convertir formato de fecha
df['fecha'] = pd.to_datetime(df['fecha'])

In [55]:
#Crear DataFrame unicamente con las variables que me interesan
df_cesta = df[['id_pedido', 'nombre_producto']]
df_cesta

,id_pedido,nombre_producto
0,1,Pepino Kirby
1,1,Bolsa de Bananas Orgánicas
2,1,Aguacate Hass Orgánico
3,2,Col Rizada Orgánica de Michigan
4,2,Zanahorias
...,...,...
4975713,3421080,Leche Entera Orgánica
4975714,3421080,Racimo de Tomates Orgánicos
4975715,3421080,Cilantro Orgánico
4975716,3421082,Fresas


In [57]:
#Agrupar los productos por pedido
df_agrupado = df_cesta.groupby('id_pedido')['nombre_producto'].apply(lambda producto: ','.join(producto))
df_agrupado

id_pedido
1          Pepino Kirby,Bolsa de Bananas Orgánicas,Aguaca...
2                 Col Rizada Orgánica de Michigan,Zanahorias
3                                   Espinacas Baby Orgánicas
5          Bolsa de Bananas Orgánicas,Frambuesas Orgánica...
10         Banana,Cilantro Orgánico,Aguacate Orgánico,Ceb...
                                 ...                        
3421077    Frambuesas Orgánicas,Calabacín Orgánico,Leche ...
3421078                       Manzanas Gala Orgánicas,Banana
3421080    Leche Entera Orgánica,Racimo de Tomates Orgáni...
3421082                                               Fresas
3421083                                               Banana
Name: nombre_producto, Length: 2060188, dtype: object

In [27]:
#Transformar los productos en columnas con 0/1
df_transacciones = df_agrupado.str.get_dummies(sep=',')
df_transacciones

,Agua con Gas de Pomelo,Aguacate Hass Orgánico,Aguacate Orgánico,Ajo Orgánico,Apio Orgánico en Ramillete Pequeño,Arándanos Orgánicos,Banana,Bolsa de Bananas Orgánicas,Calabacín Orgánico,Cebolla Amarilla Orgánica,...,Manzana Honeycrisp Orgánica,Manzanas Gala Orgánicas,Pepino Kirby,Pepino Orgánico,Racimo de Tomates Orgánicos,Rúcula Baby Orgánica,Tomates Cherry Orgánicos,Uvas Rojas sin Semillas,Zanahorias,Zanahorias Baby Orgánicas
id_pedido,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421077,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3421078,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3421080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [29]:
#Soporte para cada producto
soporte = df_transacciones.mean() * 100
soporte.sort_values(ascending=False)

Banana                                23.809381
Bolsa de Bananas Orgánicas            19.129710
Fresas Orgánicas                      13.349170
Espinacas Baby Orgánicas              12.198935
Aguacate Hass Orgánico                10.703004
Aguacate Orgánico                      8.929331
Limón Grande                           7.791279
Fresas                                 7.241960
Limones                                7.107944
Leche Entera Orgánica                  6.918301
Frambuesas Orgánicas                   6.906797
Cebolla Amarilla Orgánica              5.706324
Ajo Orgánico                           5.521778
Calabacín Orgánico                     5.302429
Arándanos Orgánicos                    5.086138
Pepino Kirby                           4.840044
Manzana Fuji Orgánica                  4.502841
Limón Orgánico                         4.422266
Tomates Cherry Orgánicos               4.266795
Manzana Honeycrisp Orgánica            4.231216
Uvas Rojas sin Semillas                4

In [31]:
#Funcion para calcular la confianza entre dos productos
def confianza(antecedente, consecuente):
  #Casos donde se compraron ambos productos
  conjunto_ac = df_transacciones[(df_transacciones[antecedente] == 1) & (df_transacciones[consecuente] == 1)]
  #Confianza = compras conjuntas / compras de producto antecedente
  return len(conjunto_ac) / df_transacciones[antecedente].sum()

In [33]:
#Funcion para calcular el lift entre dos productos
def lift(antecedente, consecuente):
  soporte_a = df_transacciones[antecedente].mean()
  soporte_c = df_transacciones[consecuente].mean()
  conteo_ac = len(df_transacciones[(df_transacciones[antecedente] == 1) & (df_transacciones[consecuente] == 1)])
  soporte_ac = conteo_ac / len(df_transacciones)
  return soporte_ac / (soporte_a * soporte_c)

In [35]:
from itertools import combinations

#Definir umbral para la confianza minima
umbral_confianza = 0.05
asociaciones = []

#Generar combinaciones de productos y calcular confianza y lift
for antecedente, consecuente in combinations(df_transacciones.columns, 2):

  #Soporte del antecedenete
  soporte_a = df_transacciones[antecedente].mean()

  #Calcular confianza
  conf = confianza(antecedente, consecuente)
  if conf >= umbral_confianza:
    asociaciones.append({
        'Antecedente': antecedente,
        'Consecuente': consecuente,
        'Soporte antecedente': round(soporte_a*100,1),
        'Confianza':round(conf*100,1),
        'Lift': round(lift(antecedente, consecuente),1)
    })

#Convertir asociaciones en DataFrame
df_asociaciones = pd.DataFrame(asociaciones)

#Ordenar las asociaciones por Lift de mayor a menor
df_asociaciones = df_asociaciones.sort_values(by='Lift', ascending=False)

#Mostrar las asociaciones
df_asociaciones

,Antecedente,Consecuente,Soporte antecedente,Confianza,Lift
224,Cilantro Orgánico,Limones,3.5,25.4,3.6
185,Cebolla Roja Orgánica,Cilantro Orgánico,3.4,12.8,3.6
61,Ajo Orgánico,Cebolla Amarilla Orgánica,5.5,20.1,3.5
106,Apio Orgánico en Ramillete Pequeño,Zanahorias,3.4,12.0,3.3
102,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,3.4,12.7,3.1
...,...,...,...,...,...
274,Crema Orgánica,Espinacas Baby Orgánicas,3.8,8.7,0.7
304,Frambuesas Orgánicas,Fresas,6.9,5.3,0.7
38,Aguacate Orgánico,Bolsa de Bananas Orgánicas,8.9,13.0,0.7
309,Frambuesas Orgánicas,Limón Grande,6.9,5.8,0.7


In [37]:
#Crear Tabala con los productos unicos y las columnas correspondientes
productos_unicos = df[['id_producto', 'id_seccion', 'id_departamento', 'nombre_producto']].drop_duplicates()
productos_unicos

,id_producto,id_seccion,id_departamento,nombre_producto
0,49683,83,4,Pepino Kirby
1,13176,24,4,Bolsa de Bananas Orgánicas
2,47209,24,4,Aguacate Hass Orgánico
3,28985,83,4,Col Rizada Orgánica de Michigan
4,17794,83,4,Zanahorias
5,21903,123,4,Espinacas Baby Orgánicas
7,27966,123,4,Frambuesas Orgánicas
9,24852,24,4,Banana
10,31717,16,4,Cilantro Orgánico
11,47766,24,4,Aguacate Orgánico


In [39]:
df_asociaciones_enriquecido =df_asociaciones.merge(productos_unicos, left_on='Antecedente', right_on='nombre_producto', how='left').drop(columns=['nombre_producto'])
df_asociaciones_enriquecido.columns = ['Antecedente', 'Consecuente', 'Soporte antecedente', 'Confianza', 'Lift', 'id_producto_antecedente', 'id_seccion_antecedente', 'id_departamento_antecedente']
df_asociaciones_enriquecido

,Antecedente,Consecuente,Soporte antecedente,Confianza,Lift,id_producto_antecedente,id_seccion_antecedente,id_departamento_antecedente
0,Cilantro Orgánico,Limones,3.5,25.4,3.6,31717,16,4
1,Cebolla Roja Orgánica,Cilantro Orgánico,3.4,12.8,3.6,8518,83,4
2,Ajo Orgánico,Cebolla Amarilla Orgánica,5.5,20.1,3.5,24964,83,4
3,Apio Orgánico en Ramillete Pequeño,Zanahorias,3.4,12.0,3.3,44359,83,4
4,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,3.4,12.7,3.1,44359,83,4
...,...,...,...,...,...,...,...,...
394,Crema Orgánica,Espinacas Baby Orgánicas,3.8,8.7,0.7,49235,53,16
395,Frambuesas Orgánicas,Fresas,6.9,5.3,0.7,27966,123,4
396,Aguacate Orgánico,Bolsa de Bananas Orgánicas,8.9,13.0,0.7,47766,24,4
397,Frambuesas Orgánicas,Limón Grande,6.9,5.8,0.7,27966,123,4


In [41]:
#Exportar a CSV
df_asociaciones_enriquecido.to_csv('asociaciones.csv', index=False, sep=';', decimal=',')